# メタ学習のPACベイズ

参考
* [PAC-Bayesian Meta-Learning: From Theory to Practice](https://arxiv.org/abs/2211.07206)



* データの分布：$\mathcal{D}$
    * そのドメインを$\mathcal{Z}$とする
    * データから$m$個の観測をサンプルする：$S=\{z_i\}_{i=1}^m$ （$S \sim \mathcal{D}^m$で略記）
* $z_i=(x_i, y_i)$：$x_i \in \mathcal{X}$は入力，$y_i \in \mathcal{Y}$は出力
* ゴール：$S$を与えられたときに，仮説集合$\mathcal{H}$から，新しい入力$x^* \mathcal{D}_x$に対して予測をする仮説$h\in \mathcal{H}$を学習したい．
    * 損失関数：$l: \mathcal{H}\times \mathcal{Z}\to \mathbb{R}$
    * 期待損失：$\mathcal{L}(h, \mathcal{D})=\mathbb{E}_{z^*\sim \mathcal{D}} l(h, z^*)$
    * 経験損失：$\hat{\mathcal{L}}(h, \mathcal{D})=\frac{1}{m}\sum^{m}_{i=1}l(h, z^*)$
* PACベイズでの仮説集合上の確率測度
    * $P\in \mathcal{M}(\mathcal{H})$：事前分布．データと独立．
    * $Q\in \mathcal{M}(\mathcal{H})$：事後分布．データに依存しても良い．
    * Gibbs誤差：$\mathcal{L}(Q, \mathcal{D})=\mathbb{E}_{h \sim Q} \mathcal{L}(h, \mathcal{D})$
        * その経験版：$\hat{\mathcal{L}}(Q, S)=\mathbb{E}_{h \sim Q} \hat{\mathcal{L}}(h, S)$
* 中心キュムラント母関数：
    * 確率変数$X$，分布$\nu$，実関数$f$について，$\Psi_{\nu, f(\cdot)}(t)=\ln \mathbb{E}_{X \sim \nu}\left[e^{t(f(X)-\mathbb{E}[f(X)])}\right]$で定義される．
    * $X\sim \nu$なる確率変数$f(X)$がどれだけその期待値から外れているかを表す量．$f(X)$がUnboundedなときの集中不等式で便利．
  

PACベイズのバウンドとして以下が知られています：

---

**Alquierのバウンド**

$\beta > 0$について，
$$
\mathcal{L}(Q, \mathcal{D}) \leq \hat{\mathcal{L}}(Q, S)+\frac{1}{\beta}\left[D_{K L}(Q \| P)+\ln \frac{1}{\delta}+\Psi(\beta, m)\right]
$$
が成り立つ．ここで，$\Psi(\beta, m)=\ln \mathbb{E}_P \mathbb{E}_{\mathcal{D}^m} \exp [\beta(\mathcal{L}(h, \mathcal{D})-\hat{\mathcal{L}}(h, S))]$．

$\Psi$は$\hat{\mathcal{L}}$についての中心キュムラント母関数．これは一般に未知だが，損失関数が特定の形（Bdoundedなど）の場合，簡単にバウンド可能．

---

Alquierのバウンドの右辺を最小にするような事後分布$Q$は[Catoniのバウンド](MATH_PAC_Bayes.ipynb)で与えられます．


## メタ学習

メタ学習の設定
* 学習者は事前分布$P$を所持し，新しいタスクに直面すると，そのデータ$S$を使って事後分布$Q$を得ます
    * これは$Q(S, P)$として書く．ここで，$Q: \mathcal{Z}^m \times \mathcal{M}(\mathcal{H}) \rightarrow \mathcal{M}(\mathcal{H})$
    * メタ学習では，良い事前分布$P$をデータによって事前に学習するのが目標です．
        * 学習用の$n$個のタスク$\left\{\tau_1, \ldots, \tau_n\right\}$
        * それぞれのタスクからは$m_i=m$個のデータをサンプルします．
        * タスクは$\tau_i \sim \mathcal{T}$からi.i.dにサンプルされます．
* hyper-prior（事前分布の事前分布）：$\mathcal{P} \in \mathcal{M}(\mathcal{M}(\mathcal{H}))$
* hyper-posterior（事前分布の事後分布）：$\mathcal{Q}$
    * この性能は次の転移誤差で評価します：$\mathcal{L}(\mathcal{Q}, \mathcal{T}):=\mathbb{E}_{P \sim \mathcal{Q}}\left[\mathbb{E}_{(\mathcal{D}, m) \sim \mathcal{T}}\left[\mathbb{E}_{S \sim \mathcal{D}^m}[\mathcal{L}(Q(S, P), \mathcal{D})]\right]\right]$
    * 経験転移誤差：$\hat{\mathcal{L}}\left(\mathcal{Q}, S_1, \ldots, S_n\right):=\mathbb{E}_{P \sim \mathcal{Q}}\left[\frac{1}{n} \sum_{i=1}^n \hat{\mathcal{L}}\left(Q\left(S_i, P\right), S_i\right)\right]$

---

**メタ学習のPACベイズバウンド**

$\delta \in (0, 1]$に対して，

$$
\begin{aligned}
\mathcal{L}(\mathcal{Q}, \mathcal{T}) \leq & \hat{\mathcal{L}}\left(\mathcal{Q}, S_1, \ldots, S_n\right)+\left(\frac{1}{\lambda}+\frac{1}{n \beta}\right) D_{K L}(\mathcal{Q} \| \mathcal{P}) \\
& +\frac{1}{n} \sum_{i=1}^n \frac{1}{\beta} \mathbb{E}_{P \sim \mathcal{Q}}\left[D_{K L}\left(Q\left(S_i, P\right) \| P\right)\right]+\underbrace{\Psi^I(\beta)+\Psi^{I I}(\lambda)+\frac{1}{\sqrt{n}} \ln \frac{1}{\delta}}_{:=C(\delta, \lambda, \beta)}
\end{aligned}
$$

が任意のhyper posterior$\mathcal{Q} \in \mathcal{M}(\mathcal{M}(\mathcal{H}))$について，確率$1-\delta$で成立します．
ここで，

---

In [6]:
import numpy as np

mu_T = 1 / 5
sigma_T = 1 / 10

d = 5
datasize = 100

w = 
x = np.random.randn(datasize, d)
y = 

(100, 5)